## Real experiments for head movements

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

In [35]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.dummy import DummyClassifier
from seqlearn.perceptron import StructuredPerceptron
import sklearn_crfsuite
from sklearn_crfsuite import metrics as skcrfmetrics
from scipy.stats import pearsonr,spearmanr

In [36]:
import glob

In [37]:
windowsize="9-11-13"
memory="middle"

PATH = "C:\\Users\\zgk261\\nomco\\"+windowsize+"\\"+memory+"\\"
PATH

'C:\\Users\\zgk261\\nomco\\9-11-13\\middle\\'

In [38]:
!dir /B C:\Users\zgk261\nomco\9-11-13\middle\

F2_M4-all-final2_primary_F2.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F2_M4-all-final2_primary_M4.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F3_F6-all-final2_primary_F3.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F3_F6-all-final2_primary_F6.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F4_F1-all-final2_primary_F1.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F4_F1-all-final2_primary_F4.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F5_F2-all-final2_primary_F2.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBac

### SKELETON TO READ FILES, SPEAKERS and PRIMARY SPEAKERS


In [39]:
speaker_set = set()

for each_file in glob.glob(PATH+"*.tab"):
    filename=each_file.split("\\")[-1]
    fileinfo=filename.split("-")
    
    primary_speaker = fileinfo[2].split("_")[-1][:-2] #fileinfo[2] = final2_primary_M1.T
    speakers = fileinfo[0]
    l_speaker,r_speaker = speakers.split("_")

    speaker_set.add(primary_speaker)

In [40]:
speaker_set

{'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6'}

### SKELETON TO READ FILES AND COSS_VALIDATE

In the following way: In each iteration, we will leave one speaker out (two files), train a model and test on those two files.

In [41]:

for speaker_on_focus in speaker_set:
    train_files = []
    test_files  = []
    
    for each_file in glob.glob(PATH+"*.tab"):
        filename=each_file.split("\\")[-1]
        fileinfo=filename.split("-")

        primary_speaker = fileinfo[2].split("_")[-1][:-2] #fileinfo[2] = final2_primary_M1.T
        speakers = fileinfo[0]
        l_speaker,r_speaker = speakers.split("_")
        if primary_speaker == l_speaker:
            secondary_speaker = r_speaker
        else:
            secondary_speaker = l_speaker

        #print (primary_speaker,l_speaker,r_speaker,speaker_on_focus)

        file_df = pd.read_csv(each_file,sep="\t")
        file_df.columns = [el.replace(primary_speaker, "PRIMA").replace(secondary_speaker, "SECON") for el in file_df.columns]

        if primary_speaker != speaker_on_focus:
            #FILES TO TRAIN!
            train_files.append(file_df)
        else:
            #FILES TO TEST!
            test_files.append(file_df)
            
        
        
    training_data = pd.concat(train_files, ignore_index=True)
    testing_data = pd.concat(test_files, ignore_index=True)
    print ("Speaker on focus:",speaker_on_focus)
    print ("Training data size:",training_data.shape)
    print ("Testing data size:",testing_data.shape)
    print ()


Speaker on focus: F5
Training data size: (181647, 28)
Testing data size: (16413, 28)

Speaker on focus: M6
Training data size: (181528, 28)
Testing data size: (16532, 28)

Speaker on focus: M4
Training data size: (181548, 28)
Testing data size: (16512, 28)

Speaker on focus: F3
Training data size: (181651, 28)
Testing data size: (16409, 28)

Speaker on focus: M1
Training data size: (180025, 28)
Testing data size: (18035, 28)

Speaker on focus: M5
Training data size: (180649, 28)
Testing data size: (17411, 28)

Speaker on focus: F4
Training data size: (181137, 28)
Testing data size: (16923, 28)

Speaker on focus: M3
Training data size: (181426, 28)
Testing data size: (16634, 28)

Speaker on focus: F1
Training data size: (181919, 28)
Testing data size: (16141, 28)

Speaker on focus: F6
Training data size: (182822, 28)
Testing data size: (15238, 28)

Speaker on focus: M2
Training data size: (182715, 28)
Testing data size: (15345, 28)

Speaker on focus: F2
Training data size: (181593, 28)


### Experiments:

It seems that in the 3-6-9 experiments considering both the past and future of frames, the MLP was not the best model, but the Logistic Regression. I think, though, that the MLP will show good performance when cross-validating with the whole data.

I will, then, try 3 classifiers:

  * SVM, because it is the default classifier that is commonly used
  * Logistic Regression, because it seems to perform the best in our initial experiments with two dialogue data
  * MLP, because when including more data, I expect to estimate a more general (and thus, better) model

In [42]:
accuracy_results = pd.DataFrame(0,columns=speaker_set,index=["MF"])
accuracy_results

,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0,0,0,0,0,0,0,0,0,0,0,0


In [43]:
import math

def map_movements(mov_class):
    if mov_class == "Nod" or mov_class == "Jerk":
        return 1
    elif mov_class == "Shake":
        return 2
    elif mov_class!=mov_class: #This returns True if the value is NAN, and it works. It might be source of errors.
        return 0
    else:
        return 3



In [44]:
accuracy_results = pd.DataFrame(0,columns=speaker_set,index=["MF"])
f1score_results = pd.DataFrame(0,columns=speaker_set,index=["MF"])
f1score_resultsw = pd.DataFrame(0,columns=speaker_set,index=["MF"])
#f1score_resultsb = pd.DataFrame(0,columns=speaker_set,index=["MF"])
#f1_score(y_true, y_pred, average='micro')

for speaker_on_focus in speaker_set:
    train_files = []
    test_files  = []
    
    for each_file in glob.glob(PATH+"*.tab"):
        filename=each_file.split("\\")[-1]
        fileinfo=filename.split("-")

        primary_speaker = fileinfo[2].split("_")[-1][:-2] #fileinfo[2] = final2_primary_M1.T
        speakers = fileinfo[0]
        l_speaker,r_speaker = speakers.split("_")
        if primary_speaker == l_speaker:
            secondary_speaker = r_speaker
        else:
            secondary_speaker = l_speaker

        #print (primary_speaker,l_speaker,r_speaker,speaker_on_focus)

        file_df = pd.read_csv(each_file,sep="\t")
        file_df.columns = [el.replace(primary_speaker, "PRIMA").replace(secondary_speaker, "SECON") for el in file_df.columns]

        if primary_speaker != speaker_on_focus:
            #FILES TO TRAIN!
            train_files.append(file_df)
        else:
            #FILES TO TEST!
            test_files.append(file_df)
            
        
        
    training_data = pd.concat(train_files, ignore_index=True)
    testing_data = pd.concat(test_files, ignore_index=True)
    print ("Speaker on focus:",speaker_on_focus)
    print ("Training data size:",training_data.shape)
    print ("Testing data size:",testing_data.shape)
    print ()
    
    
    
    
    
    
    
    #Train data
    features=['PRIMA:velocity-r', 'PRIMA:velocity-clock', 'PRIMA:velocity-x', 'PRIMA:velocity-y',
              'PRIMA:acceleration-r', 'PRIMA:acceleration-clock', 'PRIMA:acceleration-x',
              'PRIMA:acceleration-y', 'PRIMA:jerk-r', 'PRIMA:jerk-clock', 'PRIMA:jerk-x', 'PRIMA:jerk-y'
             ]

    feats_learning = training_data[features]
    X = feats_learning.dropna() #REMEMBER THIS!!!! We're ignoring any row that has a NaN value.
    scaler = MinMaxScaler((0,1))
    scaler.fit(X)
    X_norm = scaler.transform(X)
    #X_norm = X
    #We're ignoring anyrow that has a NaN value.
    Y = (training_data[(feats_learning.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].notna()*1).values #0: no movement; 1: movement
    #Y = training_data[(feats_learning.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].map(map_movements)
    
    #Test data
    feats_testing = testing_data[features]

    X_test = feats_testing.dropna() #REMEMBER THIS!!!! We're ignoring any row that has a NaN value.
    X_test_norm = scaler.transform(X_test)
    #X_test_norm = X_test
    Y_test = (testing_data[(feats_testing.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].notna()*1).values #0: no movement; 1: movement
#    Y_test =  testing_data[(feats_testing.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].map(map_movements)
    
    classifiers_to_test = [(DummyClassifier(strategy="most_frequent"),"MF"),
                           (LogisticRegression(solver="liblinear"),"LR"),
                          (LinearSVC(),"LINEARSVC"),
                          (MLPClassifier(hidden_layer_sizes=(30,30,30,30), random_state=442), "MLP")
                          ]
                        #I think that now we have to fine-tune the n_hidden_layers and their sizes (MLP)
    
    
    for clf, clfname in classifiers_to_test:
        print ("Trying classifier ",clfname,speaker_on_focus)
        clf.fit(X_norm, Y)
        y_pred = clf.predict(X_test_norm)
        accuracy_results.loc[clfname,speaker_on_focus] = accuracy_score(Y_test,y_pred)
        #f1score_resultsb.loc[clfname,speaker_on_focus] = f1_score(Y_test,y_pred)
        f1score_results.loc[clfname,speaker_on_focus] = f1_score(Y_test,y_pred, average='macro')
        f1score_resultsw.loc[clfname,speaker_on_focus] = f1_score(Y_test,y_pred, average='weighted')
    
        print (confusion_matrix(Y_test,y_pred))
        print (accuracy_score(Y_test,y_pred)) #same as f1-score (micro-averagaed)
        print (f1_score(Y_test,y_pred, average='macro'))
        print (f1_score(Y_test,y_pred, average='weighted'))
        y_pred=None
        print ()

        
accuracy_results

Speaker on focus: F5
Training data size: (181647, 28)
Testing data size: (16413, 28)

Trying classifier  MF F5


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[12154     0]
 [ 4259     0]]
0.7405105708889295
0.42545594567157907
0.6301092504347008

Trying classifier  LR F5
[[11383   771]
 [ 3167  1092]]
0.7600682385913605
0.604638808673411
0.7238804129103376

Trying classifier  LINEARSVC F5
[[11501   653]
 [ 3254  1005]]
0.7619569853165173
0.5972530568964926
0.721141920737584

Trying classifier  MLP F5
[[10741  1413]
 [ 2647  1612]]
0.7526351063181624
0.6418300342479494
0.7376571833326611

Speaker on focus: M6
Training data size: (181528, 28)
Testing data size: (16532, 28)

Trying classifier  MF M6


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[9316    0]
 [7216    0]]
0.5635131865473022
0.36041473228102755
0.4061969085325493

Trying classifier  LR M6
[[8329  987]
 [4965 2251]]
0.6399709654004355
0.5837011022023825
0.6031428120931214

Trying classifier  LINEARSVC M6
[[8423  893]
 [5166 2050]]
0.63349866924752
0.5695278170303267
0.5906071814828775

Trying classifier  MLP M6
[[8122 1194]
 [4118 3098]]
0.6786837648197436
0.6459900776262979
0.6596558306331125

Speaker on focus: M4
Training data size: (181548, 28)
Testing data size: (16512, 28)

Trying classifier  MF M4


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[10965     0]
 [ 4380     0]]
0.7145650048875856
0.4167616875712657
0.5956066346326397

Trying classifier  LR M2
[[10524   441]
 [ 3373  1007]]
0.7514499837080483
0.5960831449173908
0.7035845249149194

Trying classifier  LINEARSVC M2
[[10610   355]
 [ 3465   915]]
0.7510589768654284
0.5856689473833017
0.6980045166602284

Trying classifier  MLP M2
[[10030   935]
 [ 2828  1552]]
0.7547735418703161
0.6470301479087608
0.7307161881584472

Speaker on focus: F2
Training data size: (181593, 28)
Testing data size: (16467, 28)

Trying classifier  MF F2


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[10315     0]
 [ 6152     0]]
0.6264043237991134
0.38514674034799495
0.48251516690223695

Trying classifier  LR F2
[[8739 1576]
 [2578 3574]]
0.7477379000425093
0.7202120536999788
0.7423979391450813

Trying classifier  LINEARSVC F2
[[8928 1387]
 [2794 3358]]
0.7460982571203012
0.7132950240372806
0.7378120544366729

Trying classifier  MLP F2
[[7653 2662]
 [1744 4408]]
0.7324345661019007
0.7216244199332281
0.7354927214741169



,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0.740511,0.563513,0.648316,0.588701,0.692986,0.551088,0.591385,0.719731,0.671644,0.506497,0.714565,0.626404
LR,0.760068,0.639971,0.705426,0.659150,0.746715,0.655103,0.755303,0.757004,0.769159,0.707442,0.751450,0.747738
LINEARSVC,0.761957,0.633499,0.701672,0.652630,0.745661,0.647637,0.748862,0.756823,0.764699,0.705933,0.751059,0.746098
MLP,0.752635,0.678684,0.727531,0.685782,0.756141,0.697145,0.773799,0.753517,0.783595,0.704686,0.754774,0.732435


In [45]:
accuracy_results

,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0.740511,0.563513,0.648316,0.588701,0.692986,0.551088,0.591385,0.719731,0.671644,0.506497,0.714565,0.626404
LR,0.760068,0.639971,0.705426,0.659150,0.746715,0.655103,0.755303,0.757004,0.769159,0.707442,0.751450,0.747738
LINEARSVC,0.761957,0.633499,0.701672,0.652630,0.745661,0.647637,0.748862,0.756823,0.764699,0.705933,0.751059,0.746098
MLP,0.752635,0.678684,0.727531,0.685782,0.756141,0.697145,0.773799,0.753517,0.783595,0.704686,0.754774,0.732435


In [46]:
f1score_resultsw

,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0.630109,0.406197,0.509992,0.436293,0.567316,0.391594,0.439536,0.602434,0.539715,0.340577,0.595607,0.482515
LR,0.723880,0.603143,0.672197,0.605911,0.729751,0.601983,0.744131,0.734712,0.746630,0.706305,0.703585,0.742398
LINEARSVC,0.721142,0.590607,0.662267,0.592398,0.724272,0.588320,0.734561,0.729016,0.738233,0.703560,0.698005,0.737812
MLP,0.737657,0.659656,0.716097,0.657776,0.744250,0.672490,0.771008,0.747309,0.778221,0.704241,0.730716,0.735493


In [47]:
f1score_results

,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0.425456,0.360415,0.393320,0.370555,0.409328,0.355291,0.371616,0.418514,0.401786,0.336208,0.416762,0.385147
LR,0.604639,0.583701,0.617980,0.571541,0.666737,0.584479,0.728559,0.649305,0.695002,0.706041,0.596083,0.720212
LINEARSVC,0.597253,0.569528,0.603887,0.555548,0.656929,0.569593,0.717295,0.637740,0.682979,0.703189,0.585669,0.713295
MLP,0.641830,0.645990,0.679131,0.633117,0.687945,0.661217,0.760828,0.679781,0.743315,0.704367,0.647030,0.721624


In [48]:
accuracy_results.T.describe()

,MF,LR,LINEARSVC,MLP
count,12.000000,12.000000,12.000000,12.000000
mean,0.634612,0.721211,0.718044,0.733394
std,0.075262,0.046497,0.048615,0.034819
min,0.506497,0.639971,0.633499,0.678684
25%,0.582404,0.693857,0.689411,0.702801
50%,0.637360,0.747226,0.745880,0.742535
75%,0.698381,0.755729,0.752500,0.755115
max,0.740511,0.769159,0.764699,0.783595


In [49]:
f1score_results.T.describe()

,MF,LR,LINEARSVC,MLP
count,12.000000,12.000000,12.000000,12.000000
mean,0.387033,0.643690,0.632742,0.683848
std,0.028472,0.057922,0.060291,0.041567
min,0.336208,0.571541,0.555548,0.633117
25%,0.368020,0.593182,0.581650,0.646770
50%,0.389234,0.633643,0.620813,0.679456
75%,0.411186,0.697762,0.688032,0.708682
max,0.425456,0.728559,0.717295,0.760828
